In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import re
import matplotlib.pyplot as plt


from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

import geohash2
import pyproj
import geopandas as gpd
from shapely.geometry import Point

# Immoscout mit Selenium aufrufen und Daten scrapen

In [ ]:
# Liste der Bundesländer und Regionen
bundesland = [
    'rheinland-pfalz'
    #'bayern',
    #'hessen'
]

region = [
    'donnersbergkreis'
    #'wuerzburg-kreis',
    #'wuerzburg'
    #'frankfurt-am-main'
]

# URL für ImmobilienScout24 zur Miete von Produktionshallen
url = 'https://www.immobilienscout24.de/gewerbe-flaechen/de/' + bundesland[0] + '/' + region[0] + '/hallenproduktion-mieten/'

# Starten des Chrome-Drivers mit Selenium
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Navigieren zur URL und manuelles Lösen des Captchas
driver.get(url)
input("Bitte lösen Sie das Captcha und drücken Sie Enter, um fortzufahren...")

# Extrahieren des HTML-Inhalts der Seite
html_content = driver.page_source

# Schließen des Chrome-Drivers
driver.quit()

# Verarbeiten des HTML-Inhalts mit BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Relevante Daten mit BeautifulSoup filtern und laden

In [ ]:
# Finden und Extrahieren von Skript-Tags aus dem HTML-Inhalt
script_tags = soup.find_all('script')

# Durchsuchen der Skript-Tags nach dem relevanten Inhalt
for script_tag in script_tags:
    script_content = script_tag.string
    if script_content and 'tilesResult' in script_content:
        tiles_result_content = script_content
        break

# Überprüfen, ob der Inhalt des tilesResult-Objekts gefunden wurde
if tiles_result_content:
    start_index = tiles_result_content.find('{', tiles_result_content.find('tilesResult')) + 10
    end_index = tiles_result_content.find('numberOfHits') + len('numberOfHits') - 18
    tiles_result_data = tiles_result_content[start_index:end_index]
    
    # Ersetzen von Zeichenfolgen, um JSON-kompatiblen Inhalt zu erhalten
    json_content = re.sub(r'"[^"]*"', lambda m: m.group(0).replace(':', ''), tiles_result_data)
    json_content = re.sub(r'(\w+):', r'"\1":', json_content)
    
    # Erstellen eines DataFrames aus dem JSON-Inhalt
    df_json = pd.read_json(json_content)
    
    # Normalisieren der JSON-Daten und Erstellen eines DataFrames
    exposes_df = pd.concat([pd.json_normalize(exp) for exp in df_json['exposes']], ignore_index=True)
else:
    print("No tilesResult content found.")

In [ ]:
exposes_df.to_csv('./Immobilien_Data/Wuerzburg.csv', index=False)
warehouse_data = exposes_df

# Die Geodaten transformieren und in einen GeoDataframe laden

In [ ]:
# Funktion zur Umwandlung eines Geohash in das Koordinatensystem CSR3035
def geohash_to_csr3035(geohash):
    # Dekodierung des Geohash in Breitengrad und Längengrad
    lat, lon = geohash2.decode(geohash)
    # Definition der Koordinatensysteme WGS84 und CSR3035
    wgs84 = pyproj.Proj(init='epsg:4326')
    csr3035 = pyproj.Proj(init='epsg:3035')
    # Umwandlung der Koordinaten von WGS84 nach CSR3035
    x_csr3035, y_csr3035 = pyproj.transform(wgs84, csr3035, lon, lat)
    return x_csr3035, y_csr3035

# Funktion zur Umwandlung eines Preisstrings in einen Float-Wert
def convert_price_string_to_float(price_string):
    # Entfernen von nicht-numerischen Zeichen
    numeric_part = re.sub(r"[^\d,.]", "", price_string)
    # Ersetzen von Komma durch Punkt
    numeric_part = numeric_part.replace(",", ".")
    # Umwandlung in einen Float-Wert
    return float(numeric_part)

# Funktion zur Extraktion des größten Float-Werts aus einem Bereichsstring
def extract_largest_float(area_string):
    # Entfernen von nicht-numerischen Zeichen und Aufteilen des Strings in Teile
    area_parts = re.sub(r"[^\d\-\s]", "", area_string).split()

    # Umgang mit verschiedenen Formaten
    if len(area_parts) == 1:  # Ein-Wert-Format (3.000 m²)
        largest_value = float(area_parts[0])
    elif len(area_parts) == 3 and area_parts[1] == '-':  # Bereichsformat (5.000 - 15.000 m²)
        largest_value = max(float(area_parts[0]), float(area_parts[2]))
    else:  # Ungültiges Format
        largest_value = None

    return largest_value

# Funktion zur Extraktion des kleinsten Float-Werts aus einem Bereichsstring
def extract_smallest_float(area_string):
    # Entfernen von nicht-numerischen Zeichen und Aufteilen des Strings in Teile
    area_parts = re.sub(r"[^\d\-\s]", "", area_string).split()

    if len(area_parts) == 1:  # Ein-Wert-Format (3.000 m²)
        smallest_value = float(area_parts[0])
    elif len(area_parts) == 3 und area_parts[1] == '-':  # Bereichsformat (5.000 - 15.000 m²)
        smallest_value = min(float(area_parts[0]), float(area_parts[2]))
    else:  # Ungültiges Format
        smallest_value = None

    return smallest_value

In [ ]:
# Einlesen der Lagerhausdaten aus CSV-Dateien
warehouse_data_wuerzburg = pd.read_csv('./Immobilien_Data/Wuerzburg.csv')  # Daten aus der CSV-Datei für Würzburg einlesen
warehouse_data_wuerzburg_kreis = pd.read_csv('./Immobilien_Data/Wuerzburg-Kreis.csv')  # Daten aus der CSV-Datei für Würzburg-Kreis einlesen

# Kombinieren der Daten aus beiden CSV-Dateien
warehouse_data = pd.concat([warehouse_data_wuerzburg, warehouse_data_wuerzburg_kreis], axis=0)  # DataFrames zusammenfügen

# Extrahieren der x- und y-Koordinaten aus der 'geoGrid'-Spalte unter Verwendung einer Funktion (angenommen, 'geohash_to_csr3035' ist anderswo definiert)
warehouse_data[['x_csr3035', 'y_csr3035']] = warehouse_data['geoGrid'].apply(lambda x: pd.Series(geohash_to_csr3035(x)))

# Erstellen einer Geometriespalte unter Verwendung von Point-Objekten aus den x- und y-Koordinaten
warehouse_data['geometry'] = warehouse_data.apply(lambda row: Point(row['x_csr3035'], row['y_csr3035']), axis=1)

# Konvertieren der Daten in ein GeoDataFrame mit spezifizierter Geometriespalte und CRS
warehouses_gdf = gpd.GeoDataFrame(warehouse_data, geometry='geometry', crs='EPSG:3035')

# Zurücksetzen des Indexes, um potenzielle Probleme mit doppelten Indizes zu vermeiden
warehouses_gdf = warehouses_gdf.reset_index(drop=True)  # Index zurücksetzen und optional die alte Spalte entfernen

# Bereinigen und Verarbeiten der 'pricePerSquareMetre'-Spalte
warehouses_gdf['pricePerSquareMetre'] = warehouses_gdf['pricePerSquareMetre'].replace('', np.nan)
warehouses_gdf['pricePerSquareMetre'] = warehouses_gdf[warehouses_gdf['pricePerSquareMetre'].notna()]['pricePerSquareMetre'].apply(convert_price_string_to_float)  # Funktion auf nicht-NaN-Werte anwenden, um das Preisformat zu konvertieren
warehouses_gdf['pricePerSquareMetre'].fillna(warehouses_gdf['pricePerSquareMetre'].median(), inplace=True)  # NaN-Werte mit dem Median füllen
warehouses_gdf['pricePerSquareMetre'] = warehouses_gdf['pricePerSquareMetre'].apply(lambda x: round(x, 2))  # Preiswerte auf 2 Dezimalstellen runden

# Bereinigen und Verarbeiten der 'floorSpace'-Spalte
warehouses_gdf['floorSpace_big'] = warehouses_gdf['floorSpace'].apply(extract_largest_float)
warehouses_gdf['floorSpace_small'] = warehouses_gdf['floorSpace'].apply(extract_smallest_float)

# Erstellen einer neuen Spalte mit dem entsprechenden Preiselement
warehouses_gdf['total_price_big'] = warehouses_gdf['floorSpace_big'] * warehouses_gdf['pricePerSquareMetre'] * 12
warehouses_gdf['total_price_small'] = warehouses_gdf['floorSpace_small'] * warehouses_gdf['pricePerSquareMetre'] * 12

# Entfernen nicht benötigter Spalten
warehouses_gdf = warehouses_gdf.drop(columns=['features', 'pictureUrls'])

In [ ]:
warehouses_gdf.to_file('./Donner_Data/warehouses_donner.gpkg', driver='GPKG')